In [1]:
# peak hour and peak period volumes for typical weekday

In [2]:
import sys, os, gzip, shutil
import datetime as dt
import pandas as pd
#import geopandas as gpd
import numpy as np
import holidays
#from shapely.geometry import Point, LineString
#import matplotlib.pyplot as plt
#%matplotlib inline

In [3]:
INDIR = r'Q:\Model Projects\101_280\data\pems'
OUTDIR = r'Q:\Model Projects\101_280\data\pems'
data_type = 'station_hour'
district = 4
ca_holidays = holidays.UnitedStates(state='CA')

In [4]:
def get_dir(base, year=2020, data_type='station_hour', district=4):
    if data_type in ['station_hour','station_5min','station_meta']:
        return os.path.join(base,'D{}_Data_{}\{}'.format(district,year,data_type))
    elif data_type == 'processed_station_hour':
        return os.path.join(base,'pems','pems_station_hour_{}.h5'.format(year))
    
def get_columns(data_type, num_cols):
    if data_type == 'station_meta':
        columns = ['station','route','dir','district','county','city','state_postmile','abs_postmile','latitude','longitude',
                   'length','type','lanes','name','user_id_1','user_id_2','user_id_3','user_id_4']
    if data_type == 'station_hour':
        columns = ['timestamp', 'station', 'district', 'route', 'dir', 'lane_type', 'station_length',
                   'samples', 'obs_pct', 'total_flow', 'avg_occupancy', 'avg_speed',
                   'delay_35','delay_40','delay_45','delay_50','delay_55','delay_60']
        for i in range(0, int((num_cols - 18) / 3)):
            columns += [f'lane_{i}_flow',
                        f'lane_{i}_avg_occ',
                        f'lane_{i}_avg_speed',
                       ]
    if data_type == 'station_5min':
        columns = ['timestamp', 'station', 'district', 'route', 'dir', 'lane_type', 'station_length',
                   'samples', 'obs_pct', 'total_flow', 'avg_occupancy', 'avg_speed']
        for i in range(0, int((num_cols - 12) / 5)):
            columns += [f'lane_{i}_samples',
                        f'lane_{i}_flow',
                        f'lane_{i}_avg_occ',
                        f'lane_{i}_avg_speed',
                        f'lane_{i}_avg_obs',
                       ]
    return columns

In [5]:
locations = pd.read_csv(os.path.join(INDIR,'stable_locations.csv'), infer_datetime_format=True, parse_dates=['start_date','end_date'],)

In [6]:
locations['start_date'] = locations['start_date'].map(lambda x: x.date())
locations['end_date'] = locations['end_date'].map(lambda x: x.date())

In [7]:
threshold = 25
dfs = []
data_type = 'processed_station_hour'

typical_weekday = True
include_holidays = True
sf_only = False
continuous_only = False

if sf_only:
    locations = locations.loc[locations['county'].eq(75)]

In [8]:
if include_holidays:
    m = pd.read_csv(os.path.join(INDIR,'with_holidays','station_flow_mean_std_2005_2022.csv'))
    OUTDIR = r'Q:\Model Projects\101_280\data\pems\with_holidays'
else:
    m = pd.read_csv(os.path.join(INDIR,'station_flow_mean_std_2005_2021.csv'))

In [9]:
m.head()

,station,hour,era,mean,std,n
0,400001,0,covid-2020,314.226667,68.150980,75
1,400001,0,covid-2021,353.551724,52.354786,58
2,400001,0,pre-covid,465.872424,147.126910,1019
3,400001,1,covid-2020,234.693333,44.482907,75
4,400001,1,covid-2021,268.120690,35.710374,58


In [10]:
dfs = []
for year in np.arange(2005,2022):
    f = os.path.join(INDIR,'pems_station_hour_{}.h5'.format(year))
    print(f.split()[1])
    df = pd.read_hdf(f)
    
    #groupby_cols = ['station','route','dir','type','county','city','start_name','end_name','is_complete','year','hour']
    groupby_cols = ['station','route','dir','type','year','hour']
    flow_cols = ['total_flow','lane_0_flow','lane_1_flow','lane_2_flow','lane_3_flow',
                 'lane_4_flow','lane_5_flow','lane_6_flow','lane_7_flow']
    agg_args = {c:['mean','std'] for c in flow_cols}
    
    if typical_weekday:
        df = df.loc[df['month'].isin([3,4,5,9,10,11]) & df['day_of_week'].isin([1,2,3]) & ~df['is_holiday']]
    else:
        groupby_cols = groupby_cols[:-1] + ['month','day_of_week'] + groupby_cols[-1:]
    
    if include_holidays and not typical_weekday:
        groupby_cols = groupby_cols[:-1] + ['is_holiday'] + groupby_cols[-1:]
    else:
        df = df.loc[~df['is_holiday']]
        
    df = df.loc[df['obs_pct'].ge(threshold)]
    before = len(df)
    df = pd.merge(locations, df, on='station', suffixes=['','_obs'])
    df = df.loc[df['date'].between(df['start_date'], df['end_date'])]
    after = len(df)
    print('removing stations with unknown location before: {}, after: {}'.format(before, after))
    
    era = 'pre-covid'
    if year >= 2020:
        era = 'covid-{}'.format(year)
        
    df.insert(0,'era', era)
    
    df = pd.merge(df, m, on=['station','era','hour'])
    
    for c in flow_cols[1:]+flow_cols[:1]:
        df.loc[df['total_flow'].eq(0),c] = np.nan
        
    df['devs'] = (df['total_flow'] - df['mean']).abs() / df['std']
    
    u = df.loc[df['devs'].lt(3),flow_cols]
    
    for c in flow_cols:
        print(c)
        df['orig_{}'.format(c.replace('_flow',''))] = df[c]
        df[c] = np.nan
    
    df.update(u)
    df = df.loc[df['devs'].lt(3)]
    
    #df.to_hdf(os.path.join(INDIR,'pems_station_hour_no_outliers_{}.h5'.format(year)), 'data', )
    df.to_csv(os.path.join(OUTDIR,'pems_station_hour_no_outliers_{}.csv'.format(year)), index=False)
    
    df2 = df.groupby(groupby_cols, as_index=False).agg(agg_args)
    dfs.append(df2)


Projects\101_280\data\pems\pems_station_hour_2005.h5
removing stations with unknown location before: 1379145, after: 871514
total_flow
lane_0_flow
lane_1_flow
lane_2_flow
lane_3_flow
lane_4_flow
lane_5_flow
lane_6_flow
lane_7_flow
Projects\101_280\data\pems\pems_station_hour_2006.h5
removing stations with unknown location before: 1390234, after: 861155
total_flow
lane_0_flow
lane_1_flow
lane_2_flow
lane_3_flow
lane_4_flow
lane_5_flow
lane_6_flow
lane_7_flow
Projects\101_280\data\pems\pems_station_hour_2007.h5
removing stations with unknown location before: 1554620, after: 930782
total_flow
lane_0_flow
lane_1_flow
lane_2_flow
lane_3_flow
lane_4_flow
lane_5_flow
lane_6_flow
lane_7_flow
Projects\101_280\data\pems\pems_station_hour_2008.h5
removing stations with unknown location before: 1703117, after: 992256
total_flow
lane_0_flow
lane_1_flow
lane_2_flow
lane_3_flow
lane_4_flow
lane_5_flow
lane_6_flow
lane_7_flow
Projects\101_280\data\pems\pems_station_hour_2009.h5
removing stations with 

In [25]:
df3 = dfs[0]

In [11]:
df = pd.concat(dfs)

In [12]:
cols = []
for c1, c2 in df.columns:
    sep = '_' if len(c2) > 0 else ''
    c = '{}{}{}'.format(c1,sep,c2)
    cols.append(c)

In [13]:
old_cols = df.columns.tolist()
df.columns=cols

In [18]:
import shutil
i = 0
if os.path.exists(os.path.join(OUTDIR,'pems_cleaned_2005_2022.csv')):
    while True:
        f = os.path.join(OUTDIR,'pems_cleaned_2005_2022_v{}.csv'.format(i))
        if not os.path.exists(f):
            shutil.copy(os.path.join(OUTDIR,'pems_cleaned_2005_2022.csv'),f)
            break
        i += 1
df.to_csv(os.path.join(OUTDIR,'pems_cleaned_2005_2022.csv'))